In [1]:
import logging
import time

import scxml4py.helper
from scxml4py.engine import SCXML_Engine
from scxml4py.action import Action
from scxml4py.activity import ThreadedActivity
from scxml4py.event import Event


In [ ]:

# Installed as status listener, engine looks for "ActionStatus" action
class ActionStatus(Action):
    # implemented by the developer, stub can be generated
    def __init__(self, theData):
        Action.__init__(self, "ActionStatus", None, theData)
        self.mStatus = None
    
    def notify(self, status):
        self.mStatus = status
        logging.getLogger("scxml4py").info(">>>>Status: <" + scxml4py.helper.formatStatus(self.mStatus) + ">")
    
    # why do I need this?
    def execute(self, theCtx):
        logging.getLogger("scxml4py").info(">>>>Status: <" + scxml4py.helper.formatStatus(self.mStatus) + ">")


class ActivityOnline(ThreadedActivity):
    # implemented by the developer, stub can be generated
    def __init__(self, theEventQueue, theData):
        ThreadedActivity.__init__(self, "ActivityOnline", theEventQueue, theData)

    def run(self):
        counter = 0
        while self.isRunning() == True:
            logging.getLogger("scxml4py").info("Activity <" + self.getId() + "> is running...")
            time.sleep(1)
            counter += 1
            if counter == 5:
                self.sendInternalEvent(Event("GoOffline"))
                self.setRunning(False)
                break
            
class Data(object):
    # implemented by the developer
    # data shared between actions and activities
    def __init__(self):
        self.mSharedInfo = None
    
    def getSharedInfo(self):
        # requires mutex
        return self.mSharedInfo
    
    def setSharedInfo(self, sharedInfo):
        # requires mutex
        self.mSharedInfo = sharedInfo





In [6]:
simpleSM_string = '''<?xml version="1.0" ?>
<scxml xmlns="http://www.w3.org/2005/07/scxml" version="1.0" datamodel="ecmascript" initial="OFF">
  <state id="OFF">
    <transition event="GoOnline" target="ONLINE"/>
  </state>
  <state id="ONLINE">
    <transition event="GoOffline" target="OFF"/>
    <invoke id="ActivityOnline"/>  	
  </state>
</scxml>'''

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(threadName)s - %(module)s - %(funcName)s - %(message)s', level=logging.DEBUG)
simple_sm_data = Data()
simple_sm = SCXML_Engine(simpleSM_string, actions = [ActionStatus], activities = [ActivityOnline], data = simple_sm_data)
simple_sm.start()
simple_sm.send_signal("GoOnline", True)
status = simple_sm.get_current_status()
print (">>>>>>" + status)
time.sleep(10)
simple_sm.send_signal("GoOffline", True)
status = simple_sm.get_current_status()
print (">>>>>>" + status)
simple_sm.terminate()



2025-03-21 23:48:14,024 - INFO - MainThread - engine - __init__ - Loading SCXML model
2025-03-21 23:48:14,025 - DEBUG - MainThread - reader - parseScxml - Version: 1.0
2025-03-21 23:48:14,025 - DEBUG - MainThread - reader - parseScxml - Optional name attribute not available.
2025-03-21 23:48:14,026 - DEBUG - MainThread - reader - parseStates - Found new state: OFF
2025-03-21 23:48:14,026 - DEBUG - MainThread - reader - parseStates - Found new state: ONLINE
2025-03-21 23:48:14,026 - DEBUG - MainThread - reader - parseScxmlInitialTransition - Found initial transition to state: OFF
2025-03-21 23:48:14,027 - DEBUG - MainThread - reader - parseTransitions - Found new transition to state: ONLINE
2025-03-21 23:48:14,027 - DEBUG - MainThread - reader - parseTransitions - Found new transition to state: OFF
2025-03-21 23:48:14,027 - DEBUG - MainThread - reader - parseInvokes - Found activity ActivityOnline for state ONLINE
2025-03-21 23:48:14,028 - DEBUG - MainThread - engine - __init__ - Loaded

>>>>>>


2025-03-21 23:48:15,039 - INFO - Thread-17 (run) - 4042362149 - run - Activity <ActivityOnline> is running...
2025-03-21 23:48:16,041 - INFO - Thread-17 (run) - 4042362149 - run - Activity <ActivityOnline> is running...
2025-03-21 23:48:17,043 - INFO - Thread-17 (run) - 4042362149 - run - Activity <ActivityOnline> is running...
2025-03-21 23:48:18,044 - INFO - Thread-17 (run) - 4042362149 - run - Activity <ActivityOnline> is running...
2025-03-21 23:48:19,045 - DEBUG - Thread-17 (run) - activity - sendInternalEvent - Activity <ActivityOnline>: Triggering internal event <GoOffline>
2025-03-21 23:48:19,047 - DEBUG - Thread-16 - engine - run - Application received event = <GoOffline>
2025-03-21 23:48:19,047 - DEBUG - Thread-16 - executor - processEvent - Adding event <GoOffline> to the external queue
2025-03-21 23:48:19,048 - DEBUG - Thread-16 - executor - processEvents - Processing external event <GoOffline>
2025-03-21 23:48:19,048 - DEBUG - Thread-16 - executor - selectTransitions - Sel

>>>>>>OFF
